In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import numpy as np

In [2]:
handle = 'vaishnavi17200tripathi'

In [3]:
import requests

profile = f'https://codeforces.com/api/user.info?handles={handle}'
submission = f'https://codeforces.com/api/user.status?handle={handle}'
contest = f'https://codeforces.com/api/user.rating?handle={handle}'

prof = requests.get(profile).json()
sub = requests.get(submission).json()
con = requests.get(contest).json()

In [4]:
p = pd.DataFrame(prof['result'])
s = pd.DataFrame(sub['result'])
c = pd.DataFrame(con['result'])
c.sample()

,contestId,contestName,handle,rank,ratingUpdateTimeSeconds,oldRating,newRating
10,2111,Educational Codeforces Round 179 (Rated for Di...,vaishnavi17200tripathi,6510,1748968500,1224,1210


# Looking at the  dfs

In [5]:
c.sample()

,contestId,contestName,handle,rank,ratingUpdateTimeSeconds,oldRating,newRating
15,2123,Codeforces Round 1034 (Div. 3),vaishnavi17200tripathi,6411,1751388600,1207,1223


In [6]:
s.sample()

,id,contestId,creationTimeSeconds,relativeTimeSeconds,problem,author,programmingLanguage,verdict,testset,passedTestCount,timeConsumedMillis,memoryConsumedBytes
64,325325431,2085,1750447576,2147483647,"{'contestId': 2085, 'index': 'B', 'name': 'Ser...","{'contestId': 2085, 'participantId': 206665080...",C++17 (GCC 7-32),WRONG_ANSWER,TESTS,1,46,0


In [7]:
s['verdict'].value_counts()

verdict
WRONG_ANSWER             176
OK                       119
TIME_LIMIT_EXCEEDED       26
RUNTIME_ERROR             12
COMPILATION_ERROR         10
SKIPPED                    7
MEMORY_LIMIT_EXCEEDED      1
Name: count, dtype: int64

# Pretty print of data in 3 of the APIs

In [8]:
import pprint
val = prof['result']
pprint.pprint(val[0])

{'avatar': 'https://userpic.codeforces.org/no-avatar.jpg',
 'contribution': 0,
 'friendOfCount': 4,
 'handle': 'vaishnavi17200tripathi',
 'lastOnlineTimeSeconds': 1751902004,
 'maxRank': 'pupil',
 'maxRating': 1250,
 'rank': 'pupil',
 'rating': 1213,
 'registrationTimeSeconds': 1727975091,
 'titlePhoto': 'https://userpic.codeforces.org/no-title.jpg'}


In [9]:
import pprint
val = con['result']
pprint.pprint(val[0])

{'contestId': 2036,
 'contestName': 'Codeforces Round 984 (Div. 3)',
 'handle': 'vaishnavi17200tripathi',
 'newRating': 423,
 'oldRating': 0,
 'rank': 9517,
 'ratingUpdateTimeSeconds': 1730566200}


In [10]:
import pprint
val = sub['result']
pprint.pprint(val[100])

{'author': {'contestId': 2041,
            'ghost': False,
            'members': [{'handle': 'vaishnavi17200tripathi'}],
            'participantId': 212082103,
            'participantType': 'PRACTICE',
            'startTimeSeconds': 1732431900},
 'contestId': 2041,
 'creationTimeSeconds': 1749586874,
 'id': 323825067,
 'memoryConsumedBytes': 0,
 'passedTestCount': 4,
 'problem': {'contestId': 2041,
             'index': 'B',
             'name': 'Bowling Frame',
             'rating': 1200,
             'tags': ['binary search', 'brute force', 'math'],
             'type': 'PROGRAMMING'},
 'programmingLanguage': 'C++17 (GCC 7-32)',
 'relativeTimeSeconds': 2147483647,
 'testset': 'TESTS',
 'timeConsumedMillis': 46,
 'verdict': 'WRONG_ANSWER'}


# Working with contest

In [11]:
def rating_change_ie_improvements():
  ratings,label = rating_changes()
  rating_diff=[]
  for i in range(len(ratings)-1):
    rating_diff.append(ratings[i+1] - ratings[i])
  return rating_diff

# Creating dics

In [12]:
def dic_contest_submissions():
    dic_contest = {}   # contestId -> set of problem names
    dic_problem = {}  # problem name -> verdicts + tags + rating(if available)
    dic_tags = {}  # tags -> count

    for i in range(len(s)):
        contest_id = s['contestId'].iloc[i]
        problem = s['problem'].iloc[i]
        problem_name = problem['name']
        problem_tags = problem.get('tags', [])
        problem_verdict = s['verdict'].iloc[i]
        problem_rating = None
        try :
          problem_rating = problem['rating']
        except:
          problem_rating = None

        # Contest dictionary: set of problem names
        if contest_id in dic_contest:
            dic_contest[contest_id].add(problem_name)
        else:
            dic_contest[contest_id] = {problem_name}


        # Problem dictionary: verdict list + tags
        if problem_name in dic_problem:
            dic_problem[problem_name]['verdict'].append(problem_verdict)
        else:
            dic_problem[problem_name] = {
                'verdict': [problem_verdict],
                'tags': problem_tags,
                'rating': problem_rating
            }

        # tags -> counts
        for tag in problem_tags:
            if tag in dic_tags:
                dic_tags[tag] += 1
            else:
                dic_tags[tag] = 1

    return dic_contest, dic_problem , dic_tags


# converting imp info to dataframe

In [13]:
contest , problem , tags  = dic_contest_submissions()

In [14]:
problem_df = pd.DataFrame(problem.items(), columns=['Problem', 'Temp'])

problem_df['Verdicts'] = problem_df['Temp'].apply(lambda x: x['verdict'])
problem_df['Tags'] = problem_df['Temp'].apply(lambda x: x.get('tags', []))
problem_df['Rating'] = problem_df['Temp'].apply(lambda x: x.get('rating', None))

problem_df.drop(columns=['Temp'], inplace=True)

problem_df.head()

,Problem,Verdicts,Tags,Rating
0,A Good Problem,"[OK, WRONG_ANSWER, WRONG_ANSWER, OK, WRONG_ANS...","[bitmasks, constructive algorithms, math]",NaN
1,Line Segments,"[OK, WRONG_ANSWER, WRONG_ANSWER]","[geometry, greedy, math]",NaN
2,Add or XOR,[OK],"[greedy, math]",NaN
3,Sasha and the Casino,[WRONG_ANSWER],"[binary search, brute force, constructive algo...",1400.0
4,"Sakurako, Kosuke, and the Permutation","[OK, MEMORY_LIMIT_EXCEEDED]","[brute force, data structures, dfs and similar...",1400.0


In [15]:
problem_df['Correct'] = problem_df['Verdicts'].apply(lambda x: sum(v == 'OK' for v in x))
problem_df['TLE'] = problem_df['Verdicts'].apply(lambda x: sum(v == 'TIME_LIMIT_EXCEEDED' for v in x))
problem_df['MLE'] = problem_df['Verdicts'].apply(lambda x: sum(v == 'MEMORY_LIMIT_EXCEEDED' for v in x))

problem_df['Wrong'] = problem_df['Verdicts'].apply(len) - problem_df['Correct'] - problem_df['TLE'] - problem_df['MLE']

problem_df.drop(columns=['Verdicts'], inplace=True)


problem_df.head()

,Problem,Tags,Rating,Correct,TLE,MLE,Wrong
0,A Good Problem,"[bitmasks, constructive algorithms, math]",NaN,2,1,0,8
1,Line Segments,"[geometry, greedy, math]",NaN,1,0,0,2
2,Add or XOR,"[greedy, math]",NaN,1,0,0,0
3,Sasha and the Casino,"[binary search, brute force, constructive algo...",1400.0,0,0,0,1
4,"Sakurako, Kosuke, and the Permutation","[brute force, data structures, dfs and similar...",1400.0,1,0,1,0


## Contest df

In [16]:
contest_df = pd.DataFrame(contest.items(), columns=['ContestId', 'Problems'])
contest_df['Problems'] = contest_df['Problems'].apply(list)
contest_df['Problems_Count'] = contest_df['Problems'].apply(len)

contest_df.sample(1)

,ContestId,Problems,Problems_Count
2,2033,"[Kousuke's Assignment, Sakurako, Kosuke, and t...",2


## Tags Df

In [17]:
tags_df = pd.DataFrame(tags.items(), columns=['Tag', 'Count'])
tags_df.sample(1)

,Tag,Count
3,geometry,15


# Final DataFrame

In [18]:
problem_df.sample()

,Problem,Tags,Rating,Correct,TLE,MLE,Wrong
77,SUMdamental Decomposition,"[bitmasks, constructive algorithms, greedy, im...",1300.0,1,0,0,5


In [19]:
tags_df.sample()

,Tag,Count
21,combinatorics,6


In [20]:
contest_df.sample()

,ContestId,Problems,Problems_Count
26,2056,[Palindromic Subsequences],1


In [21]:
c.sample()

,contestId,contestName,handle,rank,ratingUpdateTimeSeconds,oldRating,newRating
14,2112,Educational Codeforces Round 180 (Rated for Di...,vaishnavi17200tripathi,6621,1750696500,1188,1207


In [22]:
p.sample()

,contribution,lastOnlineTimeSeconds,rating,friendOfCount,titlePhoto,rank,handle,maxRating,avatar,registrationTimeSeconds,maxRank
0,0,1751902004,1213,4,https://userpic.codeforces.org/no-title.jpg,pupil,vaishnavi17200tripathi,1250,https://userpic.codeforces.org/no-avatar.jpg,1727975091,pupil


# VIsuals